#Selección de caracteristicas y Reducción de Dimensionalidad

## Punto de partida

Hasta ahora hemos explorado distintas técnicas para entender y procesar nuestra data de manera que podamos pasarsela a un modelo. Esa data puede ser preprocesada (limpiada) para tener la mejor calidad posible y ser usada en un modelo de `shallow learning`.



In [11]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.metrics import r2_score
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

In [12]:
# Cargar el conjunto de datos de diabetes
diabetes = load_diabetes(scaled=False)
data = pd.DataFrame(diabetes.data, columns=diabetes.feature_names)
target = pd.DataFrame(diabetes.target, columns=['target'])
df = pd.concat([data, target], axis=1)

# Cambiar la variable "sex" por etiquetas "male" y "female"
df['sex'] = df['sex'].replace({2: 'female', 1: 'male'})

# Crear una nueva variable categórica "bmi_phase" basada en la frecuencia
bmi_bins = KBinsDiscretizer(n_bins=15, encode='ordinal', strategy='quantile')
df['bmi_phase'] = bmi_bins.fit_transform(df[['bmi']])
df["bmi_phase"] = pd.Series(map(lambda x: 'phase '+ str(int(x)),df["bmi_phase"]))
df = df.drop('bmi', axis=1)
df

,age,sex,bp,s1,s2,s3,s4,s5,s6,target,bmi_phase
0,59.0,female,101.00,157.0,93.2,38.0,4.00,4.8598,87.0,151.0,phase 13
1,48.0,male,87.00,183.0,103.2,70.0,3.00,3.8918,69.0,75.0,phase 2
2,72.0,female,93.00,156.0,93.6,41.0,4.00,4.6728,85.0,141.0,phase 12
3,24.0,male,84.00,198.0,131.4,40.0,5.00,4.8903,89.0,206.0,phase 6
4,50.0,male,101.00,192.0,125.4,52.0,4.00,4.2905,80.0,135.0,phase 3
...,...,...,...,...,...,...,...,...,...,...,...
437,60.0,female,112.00,185.0,113.8,42.0,4.00,4.9836,93.0,178.0,phase 10
438,47.0,female,75.00,225.0,166.0,42.0,5.00,4.4427,102.0,104.0,phase 6
439,60.0,female,99.67,162.0,106.6,43.0,3.77,4.1271,95.0,132.0,phase 6
440,36.0,male,95.00,201.0,125.2,42.0,4.79,5.1299,85.0,220.0,phase 11


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 442 entries, 0 to 441
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   age        442 non-null    float64
 1   sex        442 non-null    object 
 2   bp         442 non-null    float64
 3   s1         442 non-null    float64
 4   s2         442 non-null    float64
 5   s3         442 non-null    float64
 6   s4         442 non-null    float64
 7   s5         442 non-null    float64
 8   s6         442 non-null    float64
 9   target     442 non-null    float64
 10  bmi_phase  442 non-null    object 
dtypes: float64(9), object(2)
memory usage: 38.1+ KB


In [13]:
encoded_df = pd.get_dummies(df, drop_first = True)
encoded_df

,age,bp,s1,s2,s3,s4,s5,s6,target,sex_male,...,bmi_phase_phase 13,bmi_phase_phase 14,bmi_phase_phase 2,bmi_phase_phase 3,bmi_phase_phase 4,bmi_phase_phase 5,bmi_phase_phase 6,bmi_phase_phase 7,bmi_phase_phase 8,bmi_phase_phase 9
0,59.0,101.00,157.0,93.2,38.0,4.00,4.8598,87.0,151.0,False,...,True,False,False,False,False,False,False,False,False,False
1,48.0,87.00,183.0,103.2,70.0,3.00,3.8918,69.0,75.0,True,...,False,False,True,False,False,False,False,False,False,False
2,72.0,93.00,156.0,93.6,41.0,4.00,4.6728,85.0,141.0,False,...,False,False,False,False,False,False,False,False,False,False
3,24.0,84.00,198.0,131.4,40.0,5.00,4.8903,89.0,206.0,True,...,False,False,False,False,False,False,True,False,False,False
4,50.0,101.00,192.0,125.4,52.0,4.00,4.2905,80.0,135.0,True,...,False,False,False,True,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
437,60.0,112.00,185.0,113.8,42.0,4.00,4.9836,93.0,178.0,False,...,False,False,False,False,False,False,False,False,False,False
438,47.0,75.00,225.0,166.0,42.0,5.00,4.4427,102.0,104.0,False,...,False,False,False,False,False,False,True,False,False,False
439,60.0,99.67,162.0,106.6,43.0,3.77,4.1271,95.0,132.0,False,...,False,False,False,False,False,False,True,False,False,False
440,36.0,95.00,201.0,125.2,42.0,4.79,5.1299,85.0,220.0,True,...,False,False,False,False,False,False,False,False,False,False


In [15]:
# Separar los datos en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(encoded_df.drop('target', axis=1), df['target'], test_size=0.2, random_state=42)

# Entrenar el modelo de regresión lineal
regressor = LinearRegression()
regressor.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = regressor.predict(X_test)


# Calcular el R^2 ajustado
num_features = X_train.shape[1]
r2 = r2_score(y_test, y_pred)
r2_adjusted = 1 - (1 - r2) * (len(y_test) - 1) / (len(y_test) - num_features - 1)

print(f'R2 Ajustado: {r2_adjusted:.2f}')

R2 Ajustado: 0.23


# Selección de características

## Selección de Características: Filtro vs. Envoltura

La selección de características es un proceso esencial en el aprendizaje automático para identificar las características más relevantes y eliminar las que aportan poco o ninguna información al modelo. Dos enfoques comunes para realizar la selección de características son **filtro** y **envoltura**. Cada uno tiene sus propias características y aplicaciones.

## Basado en filtros

El método de selección de características de filtro se basa en la evaluación de cada característica de forma independiente, sin tener en cuenta el modelo de aprendizaje. Aquí hay algunos puntos clave:

- **Independencia:** Cada característica se evalúa por sí misma sin considerar las otras características.
- **Métricas estadísticas:** Se utilizan métricas estadísticas, como la correlación, la prueba chi-cuadrado o la prueba F, para medir la relación entre cada característica y la variable objetivo.
- **Ventajas:** Es rápido y eficiente computacionalmente, adecuado para conjuntos de datos grandes.
- **Desventajas:** No tiene en cuenta las interacciones entre características y puede seleccionar características irrelevantes si están correlacionadas con la variable objetivo.



In [16]:
from sklearn.feature_selection import SelectKBest, chi2, f_regression

# Separar las características y el objetivo
X = encoded_df  # final_df es el DataFrame transformado
y = df['target']

# Selección de características basada en chi-cuadrado (por ejemplo, seleccionar las 5 mejores características)
chi2_selector = SelectKBest(score_func=chi2, k=5)
X_chi2_selected = chi2_selector.fit_transform(X, y)

# Obtener los nombres de las características seleccionadas
selected_feature_names_chi2 = encoded_df.columns[chi2_selector.get_support()]

# Selección de características basada en la prueba F (por ejemplo, seleccionar las 5 mejores características)
f_selector = SelectKBest(score_func=f_regression, k=5)
X_f_selected = f_selector.fit_transform(X, y)

# Obtener los nombres de las características seleccionadas
selected_feature_names_f = encoded_df.columns[f_selector.get_support()]

print("Características seleccionadas basadas en chi-cuadrado:")
print(selected_feature_names_chi2)

print("\nCaracterísticas seleccionadas basadas en la prueba F:")
print(selected_feature_names_f)


Características seleccionadas basadas en chi-cuadrado:
Index(['age', 's1', 's2', 's3', 'target'], dtype='object')

Características seleccionadas basadas en la prueba F:
Index(['bp', 's3', 's4', 's5', 'target'], dtype='object')


In [17]:
encoded_df.loc[:,['age', 's1', 's2', 's3', 'target']]

,age,s1,s2,s3,target
0,59.0,157.0,93.2,38.0,151.0
1,48.0,183.0,103.2,70.0,75.0
2,72.0,156.0,93.6,41.0,141.0
3,24.0,198.0,131.4,40.0,206.0
4,50.0,192.0,125.4,52.0,135.0
...,...,...,...,...,...
437,60.0,185.0,113.8,42.0,178.0
438,47.0,225.0,166.0,42.0,104.0
439,60.0,162.0,106.6,43.0,132.0
440,36.0,201.0,125.2,42.0,220.0


In [18]:
selected_df = encoded_df.loc[:,['age', 's1', 's2', 's3', 'target']]

# Separar los datos en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(selected_df.drop('target', axis=1), df['target'], test_size=0.2, random_state=42)

# Entrenar el modelo de regresión lineal
regressor = LinearRegression()
regressor.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = regressor.predict(X_test)


# Calcular el R^2 ajustado
num_features = X_train.shape[1]
r2 = r2_score(y_test, y_pred)
r2_adjusted = 1 - (1 - r2) * (len(y_test) - 1) / (len(y_test) - num_features - 1)

print(f'R2 Ajustado: {r2_adjusted:.2f}')

R2 Ajustado: 0.33


## Basado en envoltura

El método de selección de características de envoltura implica evaluar subconjuntos de características utilizando un modelo específico. Aquí hay algunos puntos clave:

- **Modelo en uso:** Se entrena un modelo de aprendizaje automático (como regresión lineal) en subconjuntos de características diferentes y se evalúa su rendimiento.
- **Exploración exhaustiva:** Puede incluir técnicas como la eliminación hacia atrás (backward elimination), donde se comienza con todas las características y se eliminan gradualmente las menos importantes, o la selección hacia adelante (forward selection), donde se comienza con un conjunto vacío y se agregan características de manera incremental.
- **Ventajas:** Considera las interacciones entre características y puede llevar a un conjunto de características más óptimo para un modelo en particular.
- **Desventajas:** Es más costoso computacionalmente y puede ser lento, especialmente para conjuntos de datos grandes.

La elección entre el enfoque de filtro y el de envoltura depende del problema específico y de las características de los datos. El método de filtro es rápido y útil como paso inicial de preprocesamiento para eliminar características obviamente irrelevantes. El método de envoltura es más exhaustivo y se utiliza cuando se busca un rendimiento óptimo del modelo, aunque puede ser más lento y costoso computacionalmente.

In [ ]:
from sklearn.feature_selection import RFE

# Crear un modelo de regresión lineal
model = LinearRegression()

# Realizar backward elimination
rfe = RFE(estimator=model, n_features_to_select=5)  # Seleccionar las 5 mejores características
fit = rfe.fit(X, y)

# Obtener los nombres de las características seleccionadas
selected_feature_names_backward_elimination = encoded_df.columns[fit.support_]

print("Características seleccionadas mediante Backward Elimination:")
print(selected_feature_names_backward_elimination)

Características seleccionadas mediante Backward Elimination:
Index(['s4', 's5', 'target', 'bmi_phase_phase 5', 'bmi_phase_phase 6'], dtype='object')


In [ ]:
selected_df = encoded_df.loc[:,['s4', 's5', 'target', 'bmi_phase_phase 5', 'bmi_phase_phase 6']]

# Separar los datos en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(selected_df.drop('target', axis=1), df['target'], test_size=0.2, random_state=42)

# Entrenar el modelo de regresión lineal
regressor = LinearRegression()
regressor.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = regressor.predict(X_test)


# Calcular el R^2 ajustado
num_features = X_train.shape[1]
r2 = r2_score(y_test, y_pred)
r2_adjusted = 1 - (1 - r2) * (len(y_test) - 1) / (len(y_test) - num_features - 1)

print(f'R2 Ajustado: {r2_adjusted:.2f}')

R2 Ajustado: 0.37


# Reducción de dimensionalidad

## Análisis de Componentes Principales (PCA)

El **Análisis de Componentes Principales (PCA)** es una técnica fundamental en estadísticas y aprendizaje automático que se utiliza para reducir la dimensionalidad de los datos mientras se conserva la mayor cantidad posible de información. A continuación, se presentan los conceptos clave relacionados con PCA:

<img src="https://dimensionless.in/wp-content/uploads/2019/07/pca2.png">

### Reducción de Dimensionalidad

- PCA se utiliza comúnmente para abordar el problema de la alta dimensionalidad en conjuntos de datos. En muchas aplicaciones, los datos pueden tener numerosas características (dimensiones), lo que dificulta su visualización y análisis.

### Objetivo de PCA

- El objetivo principal de PCA es transformar un conjunto de datos de alta dimensionalidad en uno de menor dimensionalidad, conservando la mayor parte de la variabilidad presente en los datos originales.

### Componentes Principales

- PCA logra esto mediante la identificación de los **componentes principales**, que son combinaciones lineales de las características originales. Estos componentes capturan la mayor variabilidad en los datos.

### Proceso de PCA

El proceso típico de PCA implica los siguientes pasos:

1. **Estandarización de datos:** Antes de aplicar PCA, es importante estandarizar (centrar y escalar) las características para que tengan media cero y desviación estándar uno.

2. **Cálculo de la matriz de covarianza:** PCA calcula la matriz de covarianza de las características estandarizadas. Esta matriz describe cómo las características están relacionadas entre sí.

3. **Descomposición de la matriz de covarianza:** Luego, PCA descompone la matriz de covarianza en sus componentes principales utilizando técnicas de álgebra lineal.

4. **Selección de componentes:** Seleccionamos un número específico de componentes principales que deseamos conservar para reducir la dimensionalidad. Esto se basa en la cantidad de varianza explicada que deseamos mantener.

5. **Transformación de datos:** Finalmente, transformamos los datos originales en un nuevo conjunto de datos de menor dimensionalidad utilizando los componentes principales seleccionados.


In [ ]:
from sklearn.decomposition import PCA

X = encoded_df.drop('target', axis=1)

pca = PCA(6)
X_pca = pca.fit_transform(X)
df_pca = pd.DataFrame(data=X_pca)

print("DataFrame con Reducción de Dimensión utilizando PCA:")
print(df_pca.head())

DataFrame con Reducción de Dimensión utilizando PCA:
           0          1          2          3          4         5
0 -37.195271 -17.965289  -3.610698  -7.168584  -7.198449  3.545117
1 -15.613830  22.276664  13.606723  -5.670229 -13.944030 -0.161135
2 -37.503412 -16.683949  -0.363517 -22.470757  -6.941758  2.774591
3  14.119658  15.748912 -22.452161  13.241667   3.835168  2.861798
4   8.150311   1.648605  -0.303782  -0.132254 -14.376614 -2.415787


In [ ]:
# Separar los datos en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(df_pca, df['target'], test_size=0.2, random_state=42)

# Entrenar el modelo de regresión lineal
regressor = LinearRegression()
regressor.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = regressor.predict(X_test)


# Calcular el R^2 ajustado
num_features = X_train.shape[1]
r2 = r2_score(y_test, y_pred)
r2_adjusted = 1 - (1 - r2) * (len(y_test) - 1) / (len(y_test) - num_features - 1)

print(f'R2 Ajustado: {r2_adjusted:.2f}')

R2 Ajustado: 0.32
